# Imports

In [30]:
import pandas as pd
from NseXRBL import *
from Preprocessing import *
from Sentiments import *
from feature_engineer import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

Setting Defaults

In [31]:
stock_name = 'Zomato' 
market = '.NS' # For NSE, Indian market

fillings_file_path = f'Data/{stock_name}/CF-FR-equities-ZOMATO-24-Mar-2025.csv'
financial_file_path = f"Data/{stock_name}/{stock_name}_Financials.xlsx"
stock_price_file_path = f"./Data/{stock_name}/{stock_name}.csv"
sentiment_file_path = f'Data/{stock_name}/{stock_name}_sentiments.csv'
model_data_file_path = f'Data/{stock_name}/{stock_name}_model.csv'

# 1) Data Extraction

-------------------------------------------------------------------------------------------------------------------

### Getting quarterly financial data fron NSE (National stock exchange, India)

![Alt Text](Images/Image1.png)


In [32]:
df_fillings = pd.read_csv(fillings_file_path)
df_fillings.head(3)

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46
1,Zomato Limited,Un-Audited,Non-cumulative,Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:28:59,20-Jan-2025 18:29:27,00:00:28
2,Zomato Limited,Un-Audited,Non-cumulative,Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:01:05,22-Oct-2024 19:01:37,00:00:32


Let's understand this table:- 

- COMPANY NAME : Name of the company

- AUDITED / UNAUDITED : Weather the report sent was audited or not

- CUMULATIVE / NON-CUMULATIVE : The report is Cumalative or not

- CONSOLIDATED / NON-CONSOLIDATED : The company releases 2 reports at same time, consolidated or non-consolidated. The consolidated report will have profits and expense of previous quarters included. where as non-consolidated report will have profits and expense of the particular quarter only (which we want in our case)

- IND AS/ NON IND AS : Its represent the accounting standards used in the report

- PERIOD ENDED : End date of quarter 

- RELATING TO : The report belongs to what quarter (First, Second, Third, )

- ** XBRL : XML link of the the report(in XRBL format)

- Exchange Received Time : At what time the exchange recieved report from the company

- Exchange Dissemination Time : At what time exchange published the report 

- Time Taken : difference between 'Exchange Dissemination Time' and 'Exchange Received Time'

applying preprocessing to clean the data

In [33]:
df_fillings = preprocess_fillings_NonConsolidated_csv(fillings_file_path)
df_fillings.head()

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46
3,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:00:01,22-Oct-2024 19:00:31,00:00:30
5,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2024,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,01-Aug-2024 18:59:42,01-Aug-2024 18:59:58,00:00:16
7,Zomato Limited,Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Mar-2024,Fourth Quarter,https://nsearchives.nseindia.com/corporate/xbr...,13-May-2024 21:26:06,13-May-2024 21:26:13,00:00:07
9,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2023,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,08-Feb-2024 18:33:31,08-Feb-2024 18:33:31,::


In [34]:
# Testing
Xml_Extract_Financials(df_fillings['** XBRL'].iloc[0])

,FullTag,Value
0,RevenueFromOperations,22260000000.00
1,OtherIncome,3110000000.00
2,Income,25370000000.00
3,CostOfMaterialsConsumed,0.00
4,PurchasesOfStockInTrade,0.00
5,ChangesInInventoriesOfFinishedGoodsWorkInProgr...,0.00
6,EmployeeBenefitExpense,3330000000.00
7,FinanceCosts,40000000.00
8,DepreciationDepletionAndAmortisationExpense,270000000.00
9,OtherExpenses,15990000000.00


Here we are successfully able to extract financial report from the xml link

But from this we want only few important values like, Income, Revenue, Expenses etc.

In [35]:
# Testing
Extract_Important_Financials(df_fillings['** XBRL'].iloc[0])

,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,2.537000e+10,2.226000e+10,5.740000e+09,40000000.0,270000000.0,1.963000e+10


My functions are working fine, so let's apply it to all rows

In [36]:
new_financial_data = df_fillings['** XBRL'].apply(Extract_Important_Financials)
# Concatenate results into a DataFrame
financial_df = pd.concat(new_financial_data.tolist(), axis=0).reset_index(drop=True)

# Concatenate with original df_fillings
financial_df = pd.concat([df_fillings.reset_index(drop=True), financial_df], axis=1)

financial_df.head()

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46,2.537000e+10,2.226000e+10,5.740000e+09,40000000.0,270000000.0,1.963000e+10
1,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:00:01,22-Oct-2024 19:00:31,00:00:30,2.420000e+10,2.151000e+10,4.970000e+09,40000000.0,220000000.0,1.923000e+10
2,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2024,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,01-Aug-2024 18:59:42,01-Aug-2024 18:59:58,00:00:16,2.327000e+10,2.048000e+10,4.700000e+09,40000000.0,190000000.0,1.854000e+10
3,Zomato Limited,Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Mar-2024,Fourth Quarter,https://nsearchives.nseindia.com/corporate/xbr...,13-May-2024 21:26:06,13-May-2024 21:26:13,00:00:07,2.093000e+10,1.824000e+10,3.960000e+09,40000000.0,190000000.0,1.658000e+10
4,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2023,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,08-Feb-2024 18:33:31,08-Feb-2024 18:33:31,::,2.018000e+10,1.782000e+10,3.850000e+09,50000000.0,190000000.0,1.633000e+10


In [37]:
# checking for null values
financial_df.isnull().sum()

COMPANY NAME                                   0
AUDITED / UNAUDITED                            0
CUMULATIVE / NON-CUMULATIVE                    0
CONSOLIDATED / NON-CONSOLIDATED                0
IND AS/ NON IND AS                             0
PERIOD                                         0
PERIOD ENDED                                   0
RELATING TO                                    0
** XBRL                                        0
Exchange Received Time                         0
Exchange Dissemination Time                    0
Time Taken                                     0
Total_Revenue                                  0
RevenueFromOperations                          0
ProfitBeforeTax                                0
FinanceCosts                                   0
DepreciationDepletionAndAmortisationExpense    0
Total_Expenses                                 0
dtype: int64

In [38]:
# saving results
financial_df.to_excel(financial_file_path, index=False)

-------------------------------------------------------------------------------------------------------------------

### Getting historical stock price data

Checking if the data file already exists or not

If the file does not exist, download the complete data

In [39]:
stock_price_df = preprocess_stock_price_data(stock_name, market, stock_price_file_path)
stock_price_df.sort_values('Date', ascending=False).head()

The CSV file is already up to date.


,Date,Adj Close,Close,High,Low,Open,Volume
908,2025-03-25,NaN,211.160004,225.100006,210.600006,223.800003,62162487.0
907,2025-03-24,NaN,222.800003,230.199997,222.199997,228.800003,41909431.0
906,2025-03-21,NaN,227.520004,228.850006,220.000000,220.000000,113328549.0
905,2025-03-20,NaN,224.470001,228.000000,221.869995,226.000000,61376776.0
904,2025-03-19,NaN,223.529999,227.500000,218.500000,218.500000,65519150.0


-------------------------------------------------------------------------------------------------------------------

### Getting Headlines Data for Sentimental Analysis

In [40]:
headlines_df = preprocess_sentiment_file(stock_name, market, sentiment_file_path)
headlines_df.sort_values('Date', ascending=False).head()

The CSV file is already up to date.


,Date,Articles,Aggregated Sentiment Score
1341,2025-03-25,[Zomato share price Today Live Updates : Zomat...,0.058712
1340,2025-03-24,[Zomato Share Price Highlights : Zomato closed...,0.050704
1339,2025-03-23,"[Zomato, Kondapur restaurant fined for deliver...",0.000000
1338,2025-03-22,[Zomato Renamed as Eternal Limited – Effective...,0.106506
1337,2025-03-21,"[Zomato Share Price Today Live Updates, 21 Mar...",-0.027511


Using Google News to fetch latest market news of the company by web scrapping.

In this section I will find all the news of the company since it has been listed in stock market.

In [41]:
# Find the rows where 'Articles' is empty
empty_articles_rows = headlines_df[headlines_df['Articles'].isnull()]

headlines_df.loc[empty_articles_rows.index, 'Articles'] = empty_articles_rows['Date'].apply(lambda date: get_google_news(stock_name, date))

headlines_df.sort_values('Date', ascending=False).head()


,Date,Articles,Aggregated Sentiment Score
1341,2025-03-25,[Zomato share price Today Live Updates : Zomat...,0.058712
1340,2025-03-24,[Zomato Share Price Highlights : Zomato closed...,0.050704
1339,2025-03-23,"[Zomato, Kondapur restaurant fined for deliver...",0.000000
1338,2025-03-22,[Zomato Renamed as Eternal Limited – Effective...,0.106506
1337,2025-03-21,"[Zomato Share Price Today Live Updates, 21 Mar...",-0.027511


In [42]:
# Save progress
headlines_df.to_csv(sentiment_file_path, index= False)

Now, we will mark each article with a sentiment score. 

Interpretation of Sentiment Score:
Range: Sentiment scores typically range from -1 to +1.
Negative values (below 0): Negative sentiment.
Positive values (above 0): Positive sentiment.
Around 0: Neutral or mixed sentiment.

Since, there could be multiple articles from different sources about the company, which shows different opinions of people, we will aggregate the sentiment score for each day.

In [43]:
# Rows where Aggregated Sentiment Score is empty
empty_score_rows = headlines_df[headlines_df['Aggregated Sentiment Score'].isnull() & headlines_df['Articles'].notnull()]

# Apply the function to calculate sentiment score
headlines_df.loc[empty_score_rows.index,'Aggregated Sentiment Score'] = empty_score_rows['Articles'].apply(get_aggregated_sentiment)

# Display the updated DataFrame
headlines_df.sort_values('Date', ascending=False).head()

,Date,Articles,Aggregated Sentiment Score
1341,2025-03-25,[Zomato share price Today Live Updates : Zomat...,0.058712
1340,2025-03-24,[Zomato Share Price Highlights : Zomato closed...,0.050704
1339,2025-03-23,"[Zomato, Kondapur restaurant fined for deliver...",0.000000
1338,2025-03-22,[Zomato Renamed as Eternal Limited – Effective...,0.106506
1337,2025-03-21,"[Zomato Share Price Today Live Updates, 21 Mar...",-0.027511


Here, I was able to successfully score each article. And then was able to find its aggregated Sentiment score.

eg, 
What 0.0759 Means:
A score of 0.0759 is slightly positive, indicating that the overall sentiment of the article titles in that row is mildly optimistic or slightly favorable. However, it is not a strong positive sentiment. This suggests:

The articles may contain neutral to mildly positive language.
The content could be balanced, with both positive and negative aspects discussed, but with a slight lean towards positivity.

In [44]:
# Save progress
headlines_df.to_csv(sentiment_file_path, index= False)

Future scope for optimization...

"*To add this feature into our model, which should aggreate it further more considering the market holidays.* 
*Eg, A company news came on saturday, sunday and monday, will have a impact on its stock on monday. or holidays like chritsmas etc.*"

-------------------------------------------------------------------------------------------------------------------

# 2) Feature Engineering & Data Modeling

Loading the Dataset

In [45]:
financials_data = pd.read_excel(financial_file_path)
stock_data = pd.read_csv(stock_price_file_path)
sentiments_data = pd.read_csv(sentiment_file_path)

## Financial data

In [46]:
financials_data.head(3)

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46,25370000000,22260000000,5740000000,40000000,270000000,19630000000
1,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:00:01,22-Oct-2024 19:00:31,00:00:30,24200000000,21510000000,4970000000,40000000,220000000,19230000000
2,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2024,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,01-Aug-2024 18:59:42,01-Aug-2024 18:59:58,00:00:16,23270000000,20480000000,4700000000,40000000,190000000,18540000000


List of feature I will add in this:-

1) **Operating Margin :** `ProfitBeforeTax / Total_Revenue`

2) **EBITDA :** `ProfitBeforeTax + FinanceCosts + DepreciationDepletionAndAmortisationExpense`

3) **EBITDA Margin :** `EBITDA / Total_Revenue`

4) **Profit Margin :** `ProfitBeforeTax / Total_Revenue`

5) **Revenue Contribution :** `RevenueFromOperations / Total_Revenue`

6) **Expense Ratio :** `Total_Expenses / Total_Revenue`

7) **Depreciation Ratio :** `DepreciationDepletionAndAmortisationExpense / Total_Revenue`

8) **FinanceCost Ratio :** `FinanceCosts / Total_Revenue`

In [47]:
financials_data = add_financial_ratios(financials_data)
financials_data['Published_Date'] = pd.to_datetime(financials_data['Exchange Dissemination Time'], 
                                                            format= '%d-%b-%Y %H:%M:%S').dt.strftime('%Y-%m-%d')
financials_data.head(3)

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,...,Total_Expenses,Operating_Margin,EBITDA,EBITDA_Margin,Profit_Margin,Revenue_Contribution,Expense_Ratio,Depreciation_Ratio,FinanceCost_Ratio,Published_Date
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,...,19630000000,0.226251,6050000000,0.238471,0.226251,0.877414,0.773749,0.010642,0.001577,2025-01-20
1,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:00:01,...,19230000000,0.205372,5230000000,0.216116,0.205372,0.888843,0.794628,0.009091,0.001653,2024-10-22
2,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2024,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,01-Aug-2024 18:59:42,...,18540000000,0.201977,4930000000,0.211861,0.201977,0.880103,0.796734,0.008165,0.001719,2024-08-01


### Stock price data

In [48]:
stock_data.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-07-23,126.000000,126.000000,138.899994,115.000000,116.000000,694895290.0
1,2021-07-26,140.649994,140.649994,143.750000,125.300003,126.349998,249723854.0
2,2021-07-27,132.899994,132.899994,147.800003,127.750000,141.699997,240341900.0


List of feature I will add in this:-

1) **Daily Return :** `Closing price percentage change`

2) **Intraday Range (Volatility Indicator) :** `Day High - Day Low`

3) **Close/Open :** `Closing price / Opening price `

4) **High/Low :** `ProfitBeforeTax / Total_Revenue`

5) **Price Movement Direction (Up/Down) :** `If Close > Open means upward direction (1), else downward (0)`

6) **Rolling Statistics (Moving Averages) :** `SMA 10 and SMA 30`

7) **Rolling Volatility :** `Volatility 10 and Momentum 5D`

8) **Volume-Based Indicators :** `Volume_Change and Volume_Avg_10`

9) **Relative Strength Index (RSI) :** `RSI using talib library`

In [49]:
stock_data = add_price_features(stock_data)
stock_data.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume,Daily_Return,Intraday_Range,Close/Open,High/Low,Direction,SMA_10,SMA_30,Volatility_10,Momentum_5D,Volume_Change,Volume_Avg_10,RSI
0,2021-07-23,126.000000,126.000000,138.899994,115.000000,116.000000,694895290.0,NaN,23.899994,1.086207,1.207826,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-26,140.649994,140.649994,143.750000,125.300003,126.349998,249723854.0,0.116270,18.449997,1.113178,1.147247,1,NaN,NaN,NaN,NaN,-0.640631,NaN,NaN
2,2021-07-27,132.899994,132.899994,147.800003,127.750000,141.699997,240341900.0,-0.055101,20.050003,0.937897,1.156947,0,NaN,NaN,NaN,NaN,-0.037569,NaN,NaN


### Sentimental data

In [50]:
sentiments_data.head()

,Date,Articles,Aggregated Sentiment Score
0,2021-07-23,['Zomato IPO catapults founder towards list of...,0.062067
1,2021-07-24,['Zomatos Deepinder Goyal Is Now A Billionaire...,0.076988
2,2021-07-25,['Rally in Zomato can be sustainable in near t...,0.060000
3,2021-07-26,['Zomato share gains for second straight sessi...,0.080000
4,2021-07-27,"[""Inside Zomato and Universal Music Group's re...",0.058333


No more feature can be derived from this

## Date Modeling

Merging all the data in one dataframe

In [51]:
# Converting Date columns to pandas datetime
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
sentiments_data['Date'] = pd.to_datetime(sentiments_data['Date'])
financials_data['Published_Date'] = pd.to_datetime(financials_data['Published_Date'])

Since the report can be published in weekend or national holiday when the stock market is closed, It would difficult to merge them as it is. 

So, to solve this I will Expand each row into 3 rows (D, D+1, D+2)

In [52]:
Days = 2 # Offset
temp = financials_data.copy()

for i in range(Days):
    temp['Published_Date'] = temp['Published_Date']  + pd.Timedelta(days=1)
    financials_data = pd.concat([temp, financials_data])

financials_data.sort_values('Published_Date').head(3)

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,...,Total_Expenses,Operating_Margin,EBITDA,EBITDA_Margin,Profit_Margin,Revenue_Contribution,Expense_Ratio,Depreciation_Ratio,FinanceCost_Ratio,Published_Date
14,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2021,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,11-Aug-2021 11:55:25,...,11187000000,-0.351081,-2571000000,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-11
14,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2021,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,11-Aug-2021 11:55:25,...,11187000000,-0.351081,-2571000000,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-12
14,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2021,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,11-Aug-2021 11:55:25,...,11187000000,-0.351081,-2571000000,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13


Merging all the datasets

In [53]:
Model_data = pd.merge(stock_data, sentiments_data, how='left', on='Date')
Model_data = pd.merge(Model_data, financials_data, how='left', left_on='Date', right_on='Published_Date')
Model_data.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume,Daily_Return,Intraday_Range,Close/Open,...,Total_Expenses,Operating_Margin,EBITDA,EBITDA_Margin,Profit_Margin,Revenue_Contribution,Expense_Ratio,Depreciation_Ratio,FinanceCost_Ratio,Published_Date
0,2021-07-23,126.000000,126.000000,138.899994,115.000000,116.000000,694895290.0,NaN,23.899994,1.086207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,2021-07-26,140.649994,140.649994,143.750000,125.300003,126.349998,249723854.0,0.116270,18.449997,1.113178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2021-07-27,132.899994,132.899994,147.800003,127.750000,141.699997,240341900.0,-0.055101,20.050003,0.937897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [54]:
# applying forward fill to handle null values from financial data
Model_data = Model_data.ffill()
# Remove null values
Model_data = Model_data.dropna()
Model_data.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume,Daily_Return,Intraday_Range,Close/Open,...,Total_Expenses,Operating_Margin,EBITDA,EBITDA_Margin,Profit_Margin,Revenue_Contribution,Expense_Ratio,Depreciation_Ratio,FinanceCost_Ratio,Published_Date
29,2021-09-03,149.750000,149.750000,151.399994,135.600006,135.949997,113381518.0,0.088299,15.799988,1.101508,...,1.118700e+10,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13
30,2021-09-06,144.050003,144.050003,149.350006,143.250000,149.250000,34811620.0,-0.038063,6.100006,0.965159,...,1.118700e+10,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13
31,2021-09-07,140.050003,140.050003,146.850006,138.550003,144.899994,30421931.0,-0.027768,8.300003,0.966529,...,1.118700e+10,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13


One last feature we can add is 'how old the financials report published is from the trading day'. This will act as a weight because right now for eg, 1-90 days all will have same financial data in the dataset, but the impact of the report published on day 1 won't be same as of day 90 for that quarter.

In [55]:
Model_data['DaysAgoReportPublished'] = (Model_data['Date'] - Model_data['Published_Date']).dt.days
Model_data.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume,Daily_Return,Intraday_Range,Close/Open,...,Operating_Margin,EBITDA,EBITDA_Margin,Profit_Margin,Revenue_Contribution,Expense_Ratio,Depreciation_Ratio,FinanceCost_Ratio,Published_Date,DaysAgoReportPublished
29,2021-09-03,149.750000,149.750000,151.399994,135.600006,135.949997,113381518.0,0.088299,15.799988,1.101508,...,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13,21
30,2021-09-06,144.050003,144.050003,149.350006,143.250000,149.250000,34811620.0,-0.038063,6.100006,0.965159,...,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13,24
31,2021-09-07,140.050003,140.050003,146.850006,138.550003,144.899994,30421931.0,-0.027768,8.300003,0.966529,...,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13,25


In [56]:
print(f"Min days between report published and trading days: {Model_data['DaysAgoReportPublished'].min()}")
print(f"Max days between report published and trading days: {Model_data['DaysAgoReportPublished'].max()}")

Min days between report published and trading days: 0
Max days between report published and trading days: 101


In [57]:
# Save progress
Model_data.to_csv(model_data_file_path, index=False)

-------------------------------------------------------------------------------------------------------------------

# 3) Feature Selection

In [58]:
Model_data = pd.read_csv(model_data_file_path)
Model_data.head(3)

,Date,Adj Close,Close,High,Low,Open,Volume,Daily_Return,Intraday_Range,Close/Open,...,Operating_Margin,EBITDA,EBITDA_Margin,Profit_Margin,Revenue_Contribution,Expense_Ratio,Depreciation_Ratio,FinanceCost_Ratio,Published_Date,DaysAgoReportPublished
0,2021-09-03,149.750000,149.750000,151.399994,135.600006,135.949997,113381518.0,0.088299,15.799988,1.101508,...,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13,21
1,2021-09-06,144.050003,144.050003,149.350006,143.250000,149.250000,34811620.0,-0.038063,6.100006,0.965159,...,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13,24
2,2021-09-07,140.050003,140.050003,146.850006,138.550003,144.899994,30421931.0,-0.027768,8.300003,0.966529,...,-0.351081,-2.571000e+09,-0.310395,-0.351081,0.915007,1.350598,0.038513,0.002173,2021-08-13,25
